## Logistic regression on medium set

In [7]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, f1_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import gc
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

In [2]:
# First, let's examine the data structure to identify column names
def examine_dataframe(df):
    """Print the structure of the dataframe to identify column names"""
    print("DataFrame columns:", df.columns.tolist())
    print("First row sample:", df.iloc[0].to_dict())
    return df.columns.tolist()


# Get the absolute path of the current notebook
notebook_path = os.path.abspath('')

# Navigate to the project root (CS182-Final-Project)
project_root = os.path.dirname(notebook_path)
os.chdir(project_root)

print("Loading data...")
train_data = pickle.load(open('data/medium_set/train_data.pkl', 'rb'))
cv_data = pickle.load(open('data/medium_set/validation_data.pkl', 'rb'))
test1_data = pickle.load(open('data/medium_set/test1_data.pkl', 'rb'))
test2_data = pickle.load(open('data/medium_set/test2_data.pkl', 'rb'))

# Examine structure of the first dataframe to understand its format
print("\nExamining training data structure:")
examine_dataframe(train_data)

print("\nLoading protein embeddings...")
protein_embeddings = pickle.load(open('data/full_dataset/embeddings/embeddings_merged.pkl', 'rb'))
print(f"Loaded {len(protein_embeddings)} protein embeddings")

print(train_data.head())
for i, (key, value) in enumerate(protein_embeddings.items()):
    if i >= 5:
        break
    print(f"Protein ID: {key}, Embedding shape: {value.shape}")


Loading data...

Examining training data structure:
DataFrame columns: ['uniprotID_A', 'uniprotID_B', 'isInteraction', 'trainTest', 'sequence_A', 'sequence_B']
First row sample: {'uniprotID_A': 'Q92529', 'uniprotID_B': 'Q9H6L4', 'isInteraction': 0, 'trainTest': 'train', 'sequence_A': 'MLPRTKYNRFRNDSVTSVDDLLHSLSVSGGGGKVSAARATPAAAPYLVSGEALRKAPDDGPGSLGHLLHKVSHLKLSSSGLRGLSSAARERAGARLSGSCSAPSLAAPDGSAPSAPRAPAMSAARKGRPGDEPLPRPPRGAPHASDQVLGPGVTYVVKYLGCIEVLRSMRSLDFSTRTQITREAISRVCEAVPGAKGAFKKRKPPSKMLSSILGKSNLQFAGMSISLTISTASLNLRTPDSKQIIANHHMRSISFASGGDPDTTDYVAYVAKDPVNRRACHILECCDGLAQDVIGSIGQAFELRFKQYLQCPTKIPALHDRMQSLDEPWTEEEGDGSDHPYYNSIPSKMPPPGGFLDTRLKPRPHAPDTAQFAGKEQTYYQGRHLGDTFGEDWQQTPLRQGSSDIYSTPEGKLHVAPTGEAPTYVNTQQIPPQAWPAAVSSAESSPRKDLFDMKPFEDALKNQPLGPVLSKAASVECISPVSPRAPDAKMLEELQAETWYQGEMSRKEAEGLLEKDGDFLVRKSTTNPGSFVLTGMHNGQAKHLLLVDPEGTIRTKDRVFDSISHLINHHLESSLPIVSAGSELCLQQPVERKQ', 'sequence_B': 'MAQKPKVDPHVGRLGYLQALVTEFQETQSQDAKEQVLANLANFAYDPSNYEYLRQLQVLDLFLDSLSEENETLVEFAIGGLCNLCPDRANKEHILHAGGVPL

In [3]:
# Function to extract features using correct column names
def extract_features_batch(data_df, embeddings_dict, batch_size=100):
    """Process data in batches and extract features by combining protein embeddings"""
    # First examine the structure to get correct column names
    columns = data_df.columns.tolist()
    
    # Determine protein ID and interaction columns
    protein_a_col = None
    protein_b_col = None
    interaction_col = None
    
    # Common column name patterns
    protein_a_patterns = ['protein_a', 'protein_id_a', 'proteinA', 'proteinIDA', 'protein_A', 'protein_id_A']
    protein_b_patterns = ['protein_b', 'protein_id_b', 'proteinB', 'proteinIDB', 'protein_B', 'protein_id_B']
    interaction_patterns = ['isInteraction', 'is_interaction', 'interaction', 'label']
    
    # Find protein ID columns
    for col in columns:
        col_lower = col.lower()
        if any(pattern.lower() in col_lower for pattern in protein_a_patterns):
            protein_a_col = col
        elif any(pattern.lower() in col_lower for pattern in protein_b_patterns):
            protein_b_col = col
        elif any(pattern.lower() in col_lower for pattern in interaction_patterns):
            interaction_col = col
    
    # If we still can't find the columns, look for any that might contain protein IDs
    if protein_a_col is None or protein_b_col is None:
        # Check the first row to see if any column contains values that match keys in embeddings_dict
        first_row = data_df.iloc[0].to_dict()
        for col, val in first_row.items():
            if isinstance(val, str) and val in embeddings_dict:
                if protein_a_col is None:
                    protein_a_col = col
                elif protein_b_col is None and col != protein_a_col:
                    protein_b_col = col
    
    if protein_a_col is None or protein_b_col is None or interaction_col is None:
        print("Column detection failed. Please specify column names manually.")
        print("Available columns:", columns)
        return None, None
    
    print(f"Using columns: Protein A = '{protein_a_col}', Protein B = '{protein_b_col}', Interaction = '{interaction_col}'")
    
    X = []
    y = []
    skipped = 0
    
    # Process in batches to save memory
    for i in tqdm(range(0, len(data_df), batch_size), desc="Extracting features"):
        batch = data_df.iloc[i:i+batch_size]
        batch_X = []
        batch_y = []
        
        for _, row in batch.iterrows():
            # Get protein IDs
            protein_A = row[protein_a_col]
            protein_B = row[protein_b_col]
            
            # Skip if embeddings are not available
            if protein_A not in embeddings_dict or protein_B not in embeddings_dict:
                skipped += 1
                if skipped < 5:  # Print first few skipped to debug
                    print(f"Skipping pair: {protein_A}, {protein_B} - not found in embeddings")
                continue
            
            # Get embeddings
            embedding_A = embeddings_dict[protein_A]
            embedding_B = embeddings_dict[protein_B]
            
            # Mean pooling over sequence length to get a fixed-size representation
            if isinstance(embedding_A, torch.Tensor):
                feat_A = embedding_A.mean(dim=0).cpu().numpy()
                feat_B = embedding_B.mean(dim=0).cpu().numpy()
            else:
                feat_A = embedding_A.mean(axis=0)
                feat_B = embedding_B.mean(axis=0)
            
            # Combine features (concatenation)
            combined_features = np.concatenate([feat_A, feat_B])
            
            batch_X.append(combined_features)
            batch_y.append(row[interaction_col])
        
        # Add batch to full dataset
        X.extend(batch_X)
        y.extend(batch_y)
        
        # Clean up memory
        del batch_X, batch_y
        gc.collect()
    
    print(f"Processed {len(X)} protein pairs. Skipped {skipped} pairs (not found in embeddings).")
    return np.array(X), np.array(y)


# Extract features
print("\nExtracting features from training data...")
X_train, y_train = extract_features_batch(train_data, protein_embeddings)

if X_train is None:  # Exit if feature extraction failed
    print("Feature extraction failed. Please check the dataframe structure.")
    exit()

print("\nExtracting features from validation data...")
X_cv, y_cv = extract_features_batch(cv_data, protein_embeddings)

# Scale features
print("\nScaling features...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_cv_scaled = scaler.transform(X_cv)





Extracting features from training data...
Using columns: Protein A = 'uniprotID_A', Protein B = 'uniprotID_B', Interaction = 'isInteraction'


Extracting features: 100%|██████████| 100/100 [00:17<00:00,  5.81it/s]


Processed 10000 protein pairs. Skipped 0 pairs (not found in embeddings).

Extracting features from validation data...
Using columns: Protein A = 'uniprotID_A', Protein B = 'uniprotID_B', Interaction = 'isInteraction'


Extracting features: 100%|██████████| 20/20 [00:03<00:00,  6.42it/s]


Processed 2000 protein pairs. Skipped 0 pairs (not found in embeddings).

Scaling features...


In [4]:
def create_features(embA, embB, method='all'):
    """Create combined features from protein embeddings"""
    if method == 'all':
        return np.concatenate([embA, embB, np.abs(embA - embB), embA * embB], axis=1)
    elif method == 'add_sub':
        return np.concatenate([embA + embB, embA - embB], axis=1)
    elif method == 'mul_only':
        return embA * embB
    elif method == 'concatenate':
        return np.concatenate([embA, embB], axis=1)
    else:
        raise ValueError(f"Unknown method: {method}")

# Test the function with your existing data
print("Testing create_features function...")
if 'X_train_scaled' in locals():
    # If you already have scaled features, let's work with the raw embeddings
    # We need to extract embeddings first
    print("You already have scaled features. Let's extract embeddings for XGBoost...")

# Since you already ran extract_features_batch, let's extract the individual embeddings
# We need to modify the extract_features_batch to return separate embeddings

def extract_embeddings_separate(data_df, embeddings_dict, batch_size=100):
    """Extract separate embeddings for protein A and B"""
    columns = data_df.columns.tolist()
    
    # Find column names (reuse your existing logic)
    protein_a_col = None
    protein_b_col = None
    interaction_col = None
    
    protein_a_patterns = ['protein_a', 'protein_id_a', 'proteinA', 'proteinIDA', 'protein_A', 'protein_id_A', 'uniprotID_A']
    protein_b_patterns = ['protein_b', 'protein_id_b', 'proteinB', 'proteinIDB', 'protein_B', 'protein_id_B', 'uniprotID_B']
    interaction_patterns = ['isInteraction', 'is_interaction', 'interaction', 'label']
    
    for col in columns:
        col_lower = col.lower()
        if any(pattern.lower() in col_lower for pattern in protein_a_patterns):
            protein_a_col = col
        elif any(pattern.lower() in col_lower for pattern in protein_b_patterns):
            protein_b_col = col
        elif any(pattern.lower() in col_lower for pattern in interaction_patterns):
            interaction_col = col
    
    if protein_a_col is None or protein_b_col is None:
        first_row = data_df.iloc[0].to_dict()
        for col, val in first_row.items():
            if isinstance(val, str) and val in embeddings_dict:
                if protein_a_col is None:
                    protein_a_col = col
                elif protein_b_col is None and col != protein_a_col:
                    protein_b_col = col
    
    print(f"Using columns: Protein A = '{protein_a_col}', Protein B = '{protein_b_col}', Interaction = '{interaction_col}'")
    
    embA_list = []
    embB_list = []
    y_list = []
    skipped = 0
    
    for i in tqdm(range(0, len(data_df), batch_size), desc="Extracting embeddings"):
        batch = data_df.iloc[i:i+batch_size]
        
        for _, row in batch.iterrows():
            protein_A = row[protein_a_col]
            protein_B = row[protein_b_col]
            
            if protein_A not in embeddings_dict or protein_B not in embeddings_dict:
                skipped += 1
                continue
            
            embedding_A = embeddings_dict[protein_A]
            embedding_B = embeddings_dict[protein_B]
            
            # Mean pooling
            if isinstance(embedding_A, torch.Tensor):
                if embedding_A.dim() == 3:  # Handle (1, seq_len, dim)
                    feat_A = embedding_A[0].mean(dim=0).cpu().numpy()
                    feat_B = embedding_B[0].mean(dim=0).cpu().numpy()
                else:  # Handle (seq_len, dim)
                    feat_A = embedding_A.mean(dim=0).cpu().numpy()
                    feat_B = embedding_B.mean(dim=0).cpu().numpy()
            else:
                if len(embedding_A.shape) == 3:
                    feat_A = embedding_A[0].mean(axis=0)
                    feat_B = embedding_B[0].mean(axis=0)
                else:
                    feat_A = embedding_A.mean(axis=0)
                    feat_B = embedding_B.mean(axis=0)
            
            embA_list.append(feat_A)
            embB_list.append(feat_B)
            y_list.append(row[interaction_col])
    
    print(f"Processed {len(embA_list)} protein pairs. Skipped {skipped} pairs.")
    return np.array(embA_list), np.array(embB_list), np.array(y_list)

# Extract separate embeddings for all datasets
print("Extracting separate embeddings for XGBoost...")
train_embA, train_embB, train_y = extract_embeddings_separate(train_data, protein_embeddings)
val_embA, val_embB, val_y = extract_embeddings_separate(cv_data, protein_embeddings)
test1_embA, test1_embB, test1_y = extract_embeddings_separate(test1_data, protein_embeddings)
test2_embA, test2_embB, test2_y = extract_embeddings_separate(test2_data, protein_embeddings)

print(f"Train embeddings: A={train_embA.shape}, B={train_embB.shape}, y={train_y.shape}")
print(f"Val embeddings: A={val_embA.shape}, B={val_embB.shape}, y={val_y.shape}")
print(f"Test1 embeddings: A={test1_embA.shape}, B={test1_embB.shape}, y={test1_y.shape}")
print(f"Test2 embeddings: A={test2_embA.shape}, B={test2_embB.shape}, y={test2_y.shape}")

Testing create_features function...
You already have scaled features. Let's extract embeddings for XGBoost...
Extracting separate embeddings for XGBoost...
Using columns: Protein A = 'uniprotID_A', Protein B = 'uniprotID_B', Interaction = 'isInteraction'


Extracting embeddings: 100%|██████████| 100/100 [00:04<00:00, 23.49it/s]


Processed 10000 protein pairs. Skipped 0 pairs.
Using columns: Protein A = 'uniprotID_A', Protein B = 'uniprotID_B', Interaction = 'isInteraction'


Extracting embeddings: 100%|██████████| 20/20 [00:01<00:00, 14.05it/s]


Processed 2000 protein pairs. Skipped 0 pairs.
Using columns: Protein A = 'uniprotID_A', Protein B = 'uniprotID_B', Interaction = 'isInteraction'


Extracting embeddings: 100%|██████████| 20/20 [00:01<00:00, 18.30it/s]


Processed 2000 protein pairs. Skipped 0 pairs.
Using columns: Protein A = 'uniprotID_A', Protein B = 'uniprotID_B', Interaction = 'isInteraction'


Extracting embeddings: 100%|██████████| 100/100 [00:04<00:00, 20.41it/s]


Processed 10000 protein pairs. Skipped 0 pairs.
Train embeddings: A=(10000, 960), B=(10000, 960), y=(10000,)
Val embeddings: A=(2000, 960), B=(2000, 960), y=(2000,)
Test1 embeddings: A=(2000, 960), B=(2000, 960), y=(2000,)
Test2 embeddings: A=(10000, 960), B=(10000, 960), y=(10000,)


## Xgboost Implementation

In [9]:
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score

# Your existing XGBoost parameters
methods = ['all']  # You can add 'add_sub', 'mul_only' later
best_score = 0
best_model = None
best_method = None
best_scaler = None

xgb_params = {
    'n_estimators': 80,
    'learning_rate': 0.05,
    'max_depth': 4,
    'min_child_weight': 5,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'tree_method': 'hist',
    'random_state': 42,
    'device': 'cuda'
}

results = {}

for method in methods:
    print(f"\nTesting method: {method}")
    
    # Create features using your existing function structure
    X_train = create_features(train_embA, train_embB, method=method)
    X_val = create_features(val_embA, val_embB, method=method)
    X_test1 = create_features(test1_embA, test1_embB, method=method)
    X_test2 = create_features(test2_embA, test2_embB, method=method)

    print(f"X_train shape: {X_train.shape}")
    print(f"X_val shape: {X_val.shape}")
    print(f"train_y shape: {train_y.shape}")
    print(f"val_y shape: {val_y.shape}")
    
    # Scale features manually (without Pipeline)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test1_scaled = scaler.transform(X_test1)
    X_test2_scaled = scaler.transform(X_test2)
    
    # Create and fit XGBoost model directly
    xgb_model = xgb.XGBClassifier(**xgb_params)
    
    # Fit the model
    print("Training XGBoost model...")
    xgb_model.fit(X_train_scaled, train_y)
    
    # Validation predictions
    val_preds = xgb_model.predict(X_val_scaled)
    val_proba = xgb_model.predict_proba(X_val_scaled)[:, 1]
    
    # Calculate metrics
    val_acc = accuracy_score(val_y, val_preds)
    val_auc = roc_auc_score(val_y, val_proba)
    val_f1 = f1_score(val_y, val_preds)
    val_precision = precision_score(val_y, val_preds)
    val_recall = recall_score(val_y, val_preds)
    
    print(f"Validation Accuracy ({method}): {val_acc:.4f}")
    print(f"Validation AUC ({method}): {val_auc:.4f}")
    print(f"Validation F1 ({method}): {val_f1:.4f}")
    print(f"Validation Precision ({method}): {val_precision:.4f}")
    print(f"Validation Recall ({method}): {val_recall:.4f}")
    
    # Store results
    results[method] = {
        'accuracy': val_acc,
        'auc': val_auc,
        'f1': val_f1,
        'precision': val_precision,
        'recall': val_recall,
        'model': xgb_model,
        'scaler': scaler,
        'X_train_scaled': X_train_scaled,
        'X_val_scaled': X_val_scaled,
        'X_test1_scaled': X_test1_scaled,
        'X_test2_scaled': X_test2_scaled
    }

    if val_acc > best_score:
        best_score = val_acc
        best_model = xgb_model
        best_method = method
        best_scaler = scaler

print(f"\nBest method: {best_method} with accuracy: {best_score:.4f}")


Testing method: all
X_train shape: (10000, 3840)
X_val shape: (2000, 3840)
train_y shape: (10000,)
val_y shape: (2000,)
Training XGBoost model...


/home/richard/miniconda3/envs/esm/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [22:28:00] WARNING: /croot/xgboost-split_1724073744422/work/src/context.cc:196: XGBoost is not compiled with CUDA support.
  warnings.warn(smsg, UserWarning)


Validation Accuracy (all): 0.5995
Validation AUC (all): 0.6374
Validation F1 (all): 0.6029
Validation Precision (all): 0.5978
Validation Recall (all): 0.6080

Best method: all with accuracy: 0.5995


In [10]:
# Test on test datasets
best_results = results[best_method]
best_xgb_model = best_results['model']

print("="*50)
print("TEST SET EVALUATION")
print("="*50)

# Test1 evaluation
test1_preds = best_xgb_model.predict(best_results['X_test1_scaled'])
test1_proba = best_xgb_model.predict_proba(best_results['X_test1_scaled'])[:, 1]

test1_acc = accuracy_score(test1_y, test1_preds)
test1_auc = roc_auc_score(test1_y, test1_proba)
test1_f1 = f1_score(test1_y, test1_preds)
test1_precision = precision_score(test1_y, test1_preds)
test1_recall = recall_score(test1_y, test1_preds)

print(f"Test1 Accuracy: {test1_acc:.4f}")
print(f"Test1 AUC: {test1_auc:.4f}")
print(f"Test1 F1 Score: {test1_f1:.4f}")
print(f"Test1 Precision: {test1_precision:.4f}")
print(f"Test1 Recall: {test1_recall:.4f}")

# Test2 evaluation
test2_preds = best_xgb_model.predict(best_results['X_test2_scaled'])
test2_proba = best_xgb_model.predict_proba(best_results['X_test2_scaled'])[:, 1]

test2_acc = accuracy_score(test2_y, test2_preds)
test2_auc = roc_auc_score(test2_y, test2_proba)
test2_f1 = f1_score(test2_y, test2_preds)
test2_precision = precision_score(test2_y, test2_preds)
test2_recall = recall_score(test2_y, test2_preds)

print(f"\nTest2 Accuracy: {test2_acc:.4f}")
print(f"Test2 AUC: {test2_auc:.4f}")
print(f"Test2 F1 Score: {test2_f1:.4f}")
print(f"Test2 Precision: {test2_precision:.4f}")
print(f"Test2 Recall: {test2_recall:.4f}")

# Print classification reports
print(f"\n{'='*20} Test1 Classification Report {'='*20}")
print(classification_report(test1_y, test1_preds))

print(f"\n{'='*20} Test2 Classification Report {'='*20}")
print(classification_report(test2_y, test2_preds))

# Summary
results_summary = {
    'best_method': best_method,
    'best_score': best_score,
    'validation_metrics': results[best_method],
    'test1_metrics': {
        'accuracy': test1_acc,
        'auc': test1_auc,
        'f1': test1_f1,
        'precision': test1_precision,
        'recall': test1_recall
    },
    'test2_metrics': {
        'accuracy': test2_acc,
        'auc': test2_auc,
        'f1': test2_f1,
        'precision': test2_precision,
        'recall': test2_recall
    }
}

print("\n" + "="*60)
print("FINAL RESULTS SUMMARY")
print("="*60)
print(f"Best Method: {best_method}")
print(f"Validation Accuracy: {results[best_method]['accuracy']:.4f}")
print(f"Test1 Accuracy: {test1_acc:.4f}")
print(f"Test2 Accuracy: {test2_acc:.4f}")
print("="*60)

TEST SET EVALUATION
Test1 Accuracy: 0.5725
Test1 AUC: 0.6104
Test1 F1 Score: 0.5799
Test1 Precision: 0.5700
Test1 Recall: 0.5900

Test2 Accuracy: 0.5657
Test2 AUC: 0.6197
Test2 F1 Score: 0.1983
Test2 Precision: 0.1189
Test2 Recall: 0.5967

==================== Test1 Classification Report ====================
              precision    recall  f1-score   support

           0       0.58      0.56      0.56      1000
           1       0.57      0.59      0.58      1000

    accuracy                           0.57      2000
   macro avg       0.57      0.57      0.57      2000
weighted avg       0.57      0.57      0.57      2000


==================== Test2 Classification Report ====================
              precision    recall  f1-score   support

           0       0.93      0.56      0.70      9100
           1       0.12      0.60      0.20       900

    accuracy                           0.57     10000
   macro avg       0.53      0.58      0.45     10000
weighted avg       0